### BUILD THE NEURAL NETWORK

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

### Get Device for Training

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


### Define the Class

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1,28,28,device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([8], device='cuda:0')


### Model Layers

In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


#### nn.Flatten

In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


#### nn.Linear

In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


#### nn.ReLU

In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.2232,  0.1625, -0.1932, -0.1563,  0.0191, -0.3870,  0.3520, -0.2005,
         -0.7174,  0.0476,  0.2228,  0.1595,  0.2814, -0.0243,  0.0648,  0.2262,
         -0.3419,  0.3442, -0.1360,  0.2608],
        [ 0.3839,  0.4056, -0.3335, -0.2889, -0.3715, -0.2373,  0.3989, -0.1555,
         -0.7528, -0.2660, -0.4031,  0.2813,  0.0879,  0.3537, -0.0842,  0.3449,
         -0.5385,  0.2421, -0.2774,  0.0534],
        [ 0.2039,  0.4331, -0.4056, -0.3766, -0.4091, -0.4782,  0.1093, -0.0538,
         -0.7543,  0.0497, -0.0253,  0.3728,  0.0672,  0.1755,  0.0746, -0.2579,
         -0.2838,  0.2979, -0.2609, -0.1841]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.2232, 0.1625, 0.0000, 0.0000, 0.0191, 0.0000, 0.3520, 0.0000, 0.0000,
         0.0476, 0.2228, 0.1595, 0.2814, 0.0000, 0.0648, 0.2262, 0.0000, 0.3442,
         0.0000, 0.2608],
        [0.3839, 0.4056, 0.0000, 0.0000, 0.0000, 0.0000, 0.3989, 0.0000, 0.0000,
         0.0000, 0.0000, 0.2813, 0.0879, 0.3537, 0.00

#### nn.Sequential

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

#### nn.Softmax

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

### Model Parameters

In [12]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[-0.0192,  0.0187, -0.0125,  ...,  0.0195, -0.0259,  0.0280],
        [ 0.0038, -0.0006,  0.0083,  ...,  0.0012, -0.0286,  0.0147]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([ 0.0126, -0.0213], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([10, 512]) | Values: tensor([[ 0.0174,  0.0395, -0.0043,  ...,  0.0025,  0.0134, -0.0177],
        [-0.0222, -0.0267,  0.0147,  ..., -0.0081, -0.0208,  0.0225]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Size: torch.Size([10]) | Values